# ICPアルゴリズム
Iterative Closest Points(ICP)アルゴリズムは，点群同士の精密位置合わせ処理に広く用いられる大変有名なアルゴリズムです．
精密位置合わせ処理とは，事前にキーポイントマッチングによる姿勢推定等のアルゴリズムによって得られた（粗い）位置姿勢を初期値として，これを微修正することによって，さらによい位置合わせ結果を得る処理のことです．

本節では，Open3Dを使ったICPの実行方法と，BeslとMckayらによって発表された原著論文に従って，ICPアルゴリズムの実装方法を解説します．

## Open3DによるICPの実行
ICPアルゴリズムでは，以下の4つのステップをループすることによって位置姿勢をアップデートします．

1. ソース（位置合わせ元）点群とターゲット点群（位置合わせ先）の対応付け
2. 剛体変換の推定
3. 物体の姿勢のアップデート
4. 収束判定（収束しない場合は1.へ戻る）


Open3DにはICPアルゴリズムが実装されているので数行のコードで実行することが可能です．まずは動作させてみましょう．

In [ ]:
import open3d as o3d
import numpy as np
import copy


pcd1 = o3d.io.read_point_cloud( "../data/bun000.pcd" )
pcd2 = o3d.io.read_point_cloud( "../data/bun045.pcd" )

pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

ソース点群$P$を変数名```pcd_s```，ターゲット点群$X$を変数名```pcd_t```としてデータを用意します．
可視化結果をわかりやすくするために，点群をvoxel_down_sampleによって間引いています．

初期状態を表示させてみましょう．

In [ ]:
# 初期状態の表示
pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_s,pcd_t])

ソース点群を緑，ターゲット点群を青に着色しています．

ICPアルゴリズムの実行には，以下の関数を使います．

In [ ]:
threshold = 0.05
trans_init = np.identity(4)
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPoint()
result = o3d.pipelines.registration.registration_icp( pcd_s, pcd_t,
                                                      threshold,
                                                      trans_init,
                                                      obj_func
                                                    )

ICPアルゴリズムの実行には```o3d.pipelines.registration.registration_icp```を使います．
関数の引数は以下の通りです．
- pcd_s：位置合わせ元の点群です．この点群の姿勢をアップデートします．
- pcd_t：位置合わせ先の点群です．
- threshold：２つの点群を対応付けするときの最大距離です．
- trans_init：pcd_sの初期姿勢です．今回は，単位行列（＝pcd_sの姿勢変換なし）を初期姿勢としました．
- obj_func：目的関数（後述）の選択肢です．

位置合わせの結果は```result```に保存されています．確認しましょう．

In [ ]:
trans_reg = result.transformation
print(trans_reg)

trans_regは次の4x4の同次変換行列です．
$ {\bf T} = \left[
    \begin{array}{cc}
      R & {\bf t} \\
      {\bf 0} & 1 \\
    \end{array}
  \right]$
  
$\bf R$は3x3回転行列，$\bf t$は3次元平行移動ベクトルです．

変換後のソース点群を```pcd_reg```を赤に着色して，結果を可視化してみましょう．

In [ ]:
# 得られた変換行列を点群に適用
pcd_reg = copy.deepcopy(pcd_s).transform(trans_reg)
pcd_reg.paint_uniform_color([1.0, 0.0, 0.0])
o3d.visualization.draw_geometries([pcd_reg,pcd_t])

図の通り，変換後のソース点群（赤）がターゲット点群（青）によく一致していることがわかります．

## ICPアルゴリズムの目的関数
上記で動作確認したICPでは，以下の目的関数を最小化する変換を求めます．

$E({\bf T}) = \Sigma_{({\bf x},{\bf p}))\in \mathcal{K}}||{\bf x}-{\bf Tp}||^2$

$\mathcal{K}$は対応点セット$\{({\bf x},{\bf q})\}$で，${\bf x},{\bf q}$は$X, P$の点です．

これは，対応点間の二乗距離を使って距離を求めていることから，point-to-pointと呼ばれています．
クラスTransformationEstimationPointToPointを指定することによって，point-to-pointのICPを実行することができます．

この他にも，目的関数として，point-to-planeと呼ばれる関数が存在します．

$E({\bf T}) = \Sigma_{({\bf x},{\bf p}))\in \mathcal{K}}(({\bf x}-{\bf Tp})\cdot{\bf n_x})^2$

$\bf n_x$は点$\bf x$の法線ベクトルです．この目的関数を利用するICPでは，$\bf x$を通る法線$\bf n_x$の平面を仮定し，その平面と$\bf p$間の距離を最小化することによって，変換行列を求めます．
多くの場合で通常のICPよりも少ない繰り返し回数で収束することが知られています．しかしながら，法線付きの点群を扱わなければならないことに注意しましょう．
```obj_func```を以下のように設定することによって，point-to-planeの目的関数を利用することができます．
``` python
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPlane()
```

In [ ]:
threshold = 0.05
trans_init = np.identity(4)
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPlane()
result = o3d.pipelines.registration.registration_icp( pcd_s, pcd_t,
                                                      threshold,
                                                      trans_init,
                                                      obj_func
                                                    )

In [ ]:
# 得られた変換行列を点群に適用
pcd_reg = copy.deepcopy(pcd_s).transform(trans_reg)
pcd_reg.paint_uniform_color([1.0, 0.0, 0.0])
o3d.visualization.draw_geometries([pcd_reg,pcd_t])

## ICPアルゴリズムの実装
前述した通り，Open3Dを使うことによって，ICPアルゴリズムの動作を確認することができます．
ここでは，アルゴリズムの内部を理解するために，Pythonコードとして実装してみます．
実装する内容は原著論文[1] に従います．
論文によると，ICPの手順は以下の4ステップです．
- Step 1. ソース（位置合わせ元）点群とターゲット点群（位置合わせ先）の対応付け
- Step 2. 剛体変換の推定
- Step 3. 物体の姿勢のアップデート
- Step 4. 収束判定（収束しない場合は1.へ戻る）

以下では，このステップごとに実装します．

[1] Paul J Besl and Neil D McKay, Method for registration of 3-D shapes, IEEE Trans on PAMI, Vol.14, No.2, pp.239-256, 1992.

In [ ]:
# 必要なパッケージの読み込み
%matplotlib inline
import open3d as o3d
import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

まずは位置合わせ対象の点群を読み込みます．先ほどと同様，ソース点群$P$を変数名```pcd_s```，ターゲット点群$X$を変数名```pcd_t```としてデータを用意し，初期状態を可視化します．

In [ ]:
pcd1 = o3d.io.read_point_cloud( "../data/bun000.pcd" )
pcd2 = o3d.io.read_point_cloud( "../data/bun045.pcd" )

pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_t, pcd_s] )

さて，論文で紹介されたICPアルゴリズムでは，物体の姿勢を同次変換行列ではなく，７つの実数値で表されるレジストレーションベクトル${\bf q}$として扱います．
これは，ICPアルゴリズムにおける姿勢計算において，Hornによって提案された手法 --対応の取れた点群同士の位置ずれ誤差を最小化する位置姿勢を計算するアルゴリズム[2]-- を適用するためです．

[2]
B. Horn, "Closed-form solution of absolute orientation using unit quaternions," J. Opt. Soc. Am. A  4, 629-642 (1987).

${\bf q}$の最初の4つの実数値${\bf q}_R = [q_0 q_1 q_2 q_3]^T$は回転成分を表現する四元数であり，
$q_0 \geq 0$かつ$q_0^2+q_1^2+q_2^2+q_3^2=1$です．
四元数から，3x3回転行列への変換は以下とおりです．


\begin{equation}
{\bf R} = \left[
    \begin{array}{ccc}
      q_0^2+q_1^2-q_2^2-q_3^2 & 2(q_1q_2-q_0q_3) & 2(q_1q_3+q_0q_2)\\
      2(q_1q_2+q_0q_3) & q_0^2+q_2^2-q_1^2-q_3^2 & 2(q_2q_3-q_0q_1)\\
      2(q_1q_3-q_0q_2) & 2(q_2q_3+q_0q_1) & q_0^2+q_3^2-q_1^2-q_2^2\\
    \end{array}
  \right]
\end{equation}

点群を剛体変換させるときは4x4同次変換行列を使うと楽なので，四元数は上記の式を使って回転行列に変換します．
後半の3つの実数値${\bf q}_t = [q_4 q_5 q_6]^T$は平行移動ベクトルです．


原著論文のICPアルゴリズムは，レジストレーションベクトルを使って以下の目的関数を最小化します．

\begin{equation}
f({\bf q}) = \frac{1}{N}\sum_{i=1}^{N}||{\bf x_i} - {\bf R}({\bf q}_R){\bf p}_i - {\bf q}_t||
\end{equation}

ここで，${\bf R}({\bf q}_R)$は，四元数${\bf q}_R$を回転行列に変換する関数です．
まず，レジストレーションベクトルを初期化します．

\begin{equation}
{\bf q} = (1, 0, 0, 0, 0, 0, 0)
\end{equation}

ICPアルゴリズムでは，この${\bf q}$の値をアップデートしながら，ソースをターゲットに位置合わせする剛体変換を算出します．
${\bf q}$の初期化と四元数から回転行列への変換は以下の通りに実装します．

In [ ]:
def quaternion2rotation( q ):
    rot = np.array([[q[0]**2+q[1]**2-q[2]**2-q[3]**2, 
                     2.0*(q[1]*q[2]-q[0]*q[3]), 
                     2.0*(q[1]*q[3]+q[0]*q[2])],
                    
                    [2.0*(q[1]*q[2]+q[0]*q[3]),
                    q[0]**2+q[2]**2-q[1]**2-q[3]**2,
                     2.0*(q[2]*q[3]-q[0]*q[1])],
                    
                    [2.0*(q[1]*q[3]-q[0]*q[2]),
                     2.0*(q[2]*q[3]+q[0]*q[1]),
                    q[0]**2+q[3]**2-q[1]**2-q[2]**2]]
                  )
    return rot

q = np.array([1.,0.,0.,0.,0.,0.,0.])
rot = quaternion2rotation(q)
print(rot)

初期値の$\bf q$は回転行列に変換すると単位行列になります．つまり，点群を回転させないことを意味します．

## Step 1. 最近傍点探索

Step1を実装しましょう．このステップの目的は，ソース点群$P_k$の各点の対応点をターゲット$X$から探すことです．$X$の点数は$P$と一致していなくても構いません．
見つかった対応点の集まりを$Y_k$とします．
$k$はICPアルゴリズムの各ステップの繰り返し回数で,$P_k, Y_k$の点数$N$は同一です．
$Y_k$に登録される点は重複することがありますが，これを許します．また，$X$の点数は$P$と一致していなくても構いません．


ここで，2点のユークリッド距離を$d({\bf r}_1, {\bf r}_2)$とします．この表記を拡張し，
特定の点$p$と，点数$N_a$で構成される点群$A = \{{\bf a}_i\}, i=1,...,N_a$の距離を以下で定義します．

\begin{equation}
d({\bf p}, A) =  \min_{ i \in \{1,...N_a\} } d({\bf p}, {\bf a}_i)
\end{equation}

このステップでは，$P_k$に含まれる全点に対して$X$との最近傍点を取り出し，これを対応点セット$Y_k$とします．
この処理を

\begin{equation}
Y_k = C(P_k,X)
\end{equation}
と表記します．
単純には全数探索すればよいのですが，高速化のために，kd-treeによる最近傍点探索を利用します．

In [ ]:
pcd_tree = o3d.geometry.KDTreeFlann(pcd_t)

idx_list = []
for i in range(len(pcd_s.points)):
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd_s.points[i], 1)
    idx_list.append(idx[0])

np_pcd_t = np.asarray(pcd_t.points)
np_pcd_y = np_pcd_t[idx_list].copy()

ICPアルゴリズムの処理のループ中では，ターゲット点群に対する最近傍探索を何度も実行します．
このため，あらかじめターゲット点群からkd-treeを構築しておきます(1行目)．

2行目以降では，ソースの点群を１点ずつクエリとして，最近傍点のインデクスをリストとして取り出す処理を実行しています．

ここで一旦，対応点セット$Y_k$を確認してみましょう．対応点を線で結んで表示します．

In [ ]:
def GetCorrespondenceLines( pcd_s, pcd_t, idx_list ):
    
    # make point cloud pair
    np_pcd_s = np.asarray(pcd_s.points)
    np_pcd_t = np.asarray(pcd_t.points)
    np_pcd_pair = np.concatenate((np_pcd_s,np_pcd_t))

    # choose pairing ids
    lines = list()
    n_points = len(pcd_s.points)
    for i in range(n_points):
        lines.append([i,n_points+idx_list[i]])

    # make lineset    
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np_pcd_pair),
        lines=o3d.utility.Vector2iVector(lines),
    )
    return line_set

line_set = GetCorrespondenceLines( pcd_s, pcd_t, idx_list )
o3d.visualization.draw_geometries([pcd_s,pcd_t,line_set])

関数```GetCorrespondenceLines```では，ソース点群，ターゲット点群，ソースに対するターゲットの対応点のインデクスリストの３つを入力として，Open3Dで可視化可能なLineSetを作成します．

緑がソース点群，青がターゲット点群です．ソース点群からターゲットの最近傍探索を実施したので，各緑点に対して最も近い青点が線で結ばれています．

## Step 2. 剛体変換の推定
$P_k, Y_k$の共分散行列を使って，2つの点群間の位置ずれ誤差が最小となる剛体変換を推定します．
まず，2つの点群$P_k, Y_k$の重心を算出します．

\begin{equation}
\vec{\mu}_p = \frac{1}{N}\sum_{i=1}^{N}{\bf p}_i , \vec{\mu}_y = \frac{1}{N}\sum_{i=1}^{N}{\bf y}_i 
\end{equation}

重心位置は，点群を```open3d.geometry.PointCloud```で扱っているときは```get_center()```で取り出すことが出来ますが，後々の計算の都合で，点群データをnumpy配列に変換してから重心を計算します．

In [ ]:
np_pcd_s = np.asarray(pcd_s.points)

mu_s = np_pcd_s.mean(axis=0)
mu_y = np_pcd_y.mean(axis=0)

次に以下の式で表す共分散行列$\Sigma_{py}$を計算します．
\begin{equation}
\Sigma_{py} = \frac{1}{N}\sum_{i=1}^{N}[({\bf p}_i-\vec{\mu}_p)({\bf y}_i-\vec{\mu}_y)^T] = \frac{1}{N}\sum_{i=1}^{N}[{\bf p}_i{\bf y}_i^T]-\vec{\mu}_p\vec{\mu}_y^T
\end{equation}

In [ ]:
np_pcd_s = np.asarray(pcd_s.points)
covar = np.zeros( (3,3) )
n_points = np_pcd_s.shape[0]
for i in range(n_points):
    covar += np.dot( np_pcd_s[i].reshape(-1, 1), np_pcd_y[i].reshape(1, -1) )
covar /= n_points
covar -= np.dot( mu_s.reshape(-1,1), mu_y.reshape(1,-1) )
print(covar)

次に以下の式で表す4x4の対称行列$Q(\Sigma_{py})$を作成します．

\begin{equation}
Q(\Sigma_{py}) = \left[
    \begin{array}{cc}
      tr(\Sigma_{py}) & \Delta^T \\
      \Delta & \Sigma_{py}+\Sigma_{py}^T-tr(\Sigma_{py})I_3\\
    \end{array}
  \right]
\end{equation}

$tr(\Sigma_{py})$は$\Sigma_{py}$の対角成分の総和，
$\Delta = [A_{12} A_{31} A_{12}]^T$は$A_{ij} = (\Sigma_{py}-\Sigma_{py}^T)_{ij}$で構成されるベクトル，
$I_3$は3x3単位行列です．

In [ ]:
A = covar - covar.T
delta = np.array([A[1,2],A[2,0],A[0,1]])
tr_covar = np.trace(covar)
i3d = np.identity(3)

Q = np.zeros((4,4))
Q[0,0] = tr_covar
Q[0,1:4] = delta
Q[1:4,0] = delta
Q[1:4,1:4] = covar + covar.T - tr_covar*i3d
print(Q)

$Q(\Sigma_{py})$の最大固有値に対する固有ベクトルが${\bf q}_R = [q_0 q_1 q_2 q_3]^T$に対応します．

$Q(\Sigma_{py})$の最大固有値に対する固有ベクトルが${\bf q}_R = [q_0 q_1 q_2 q_3]^T$に対応します．
固有値の計算は```np.linalg.eig()```に任せます．```w```は固有値，```v```が固有ベクトルです．
```v```において，最大固有値のインデクスの列成分が${\bf q}_R$です．取り出して回転行列に変換しましょう．

In [ ]:
w, v = LA.eig(Q)
rot = quaternion2rotation(v[:,np.argmax(w)])
print("Eigen value\n",w)
print("Eigen vector\n",v)
print("Rotation\n", rot)

固有値の計算は```np.linalg.eig()```に任せます．```w```は固有値，```v```が固有ベクトルです．
```v```において，最大固有値のインデクスの列成分が${\bf q}_R$です．取り出して回転行列に変換しましょう．

平行移動成分は以下の式で計算できます．
\begin{equation}
{\bf q}_t = \vec{\mu}_y - {\bf R}({\bf q}_R)\vec{\mu}_p
\end{equation}

In [ ]:
trans = mu_y - np.dot(rot,mu_s)

## Step 3. 物体の姿勢のアップデート
ここまででレジストレーションベクトル$\bf q$がわかりました．実際に点群を剛体変換させるときは4x4同次変換行列が簡単です．
最終的な同次変換行列は以下です．

In [ ]:
transform = np.identity(4)
transform[0:3,0:3] = rot.copy()
transform[0:3,3] = trans.copy()
print("Transformation\n", transform)

この変換行列を使えばソース点群はターゲット点群に近づくはずです．可視化してみましょう．
今回の変換で作成する点群は赤にします．

In [ ]:
pcd_s2 = copy.deepcopy(pcd_s) 
pcd_s2.transform(transform)
pcd_s2.paint_uniform_color([1.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_t, pcd_s, pcd_s2] )

もともとの緑の点群が変換され，位置合わせ対象の青の点群に近づいていることがわかりました．

## Step 4. 収束判定

後は，```pcd_s2```を$P_{k}$として，終了条件を満たすまでStep1-Step3を繰り返します．
終了条件には以下のものがよく使われます．
- ソース点群とターゲット点群の二乗誤差がしきい値以下になる
- 指定しておいた最大の繰り返し回数に到達する
- k-1番目とk番目の剛体変換の差が少ない

最初の条件は，あらかじめ決めた位置合わせ誤差以内に収束したかどうかをチェックすることができるので，
この条件に当てはまって繰り返し演算が終了したときは，位置合わせが成功したことになります．
残りの２つの条件は，決められた回数内での繰り返し演算で，位置合わせ誤差が十分小さくならなかった場合や，
位置合わせ誤差の改善が起きなかった場合に該当します．
この条件に当てはまって終了した場合は，所望の結果が得られたのかどうかを確認する必要があります．


```icp_registration.py```は，ここまでの処理をクラスとして実装したサンプルコードです．

In [ ]:
from icp_registration import ICPRegistration

クラス```ICPRegistration```は，ソース点群とターゲット点群を入力し，
ICPの繰り返し処理は```registration()```によって実行されます．
```set_...```の関数で終了条件を指定しています．

In [ ]:
reg = ICPRegistration(pcd_s, pcd_t)
reg.set_n_iterations( 100 )
reg.set_th_distance( 0.003 )
reg.set_th_ratio( 0.999 )
pcd_reg = reg.registration()

繰り返し演算中の位置合わせ誤差の推移は，```reg.d```によって確認することができます．

In [ ]:
plt.ylabel("Distance [m/pts.]")
plt.xlabel("# of iterations")
plt.plot(reg.d)
plt.savefig("reg_error.png")

ICPアルゴリズムの繰り返し演算は，誤差が単調減少することが保証されています．
グラフから，このことを確認することができました．

最終的な位置合わせ結果の点群は```pcd_reg```です．

In [ ]:
pcd_reg.paint_uniform_color([1.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_t, pcd_reg] )

Open3DのICPアルゴリズムと同様に，位置合わせできていることがわかりました．

最後に，各ステップでの位置合わせの様子をアニメーションとして可視化してみましょう．

In [ ]:
from icp_registration import visualize_icp_progress
    
visualize_icp_progress( reg )

0.3秒おきに画面が更新されて，徐々に位置合わせされている様子を確認することができました．